In [1]:
# this is the main module. I will use this to gate running scripts as I develop. My aim is to develop clear well documented readable code.
from input import *
from tensorflow.keras.datasets import mnist    

In [2]:

# Exemplifies how to use the filter module 
import numpy as np
lambdas = [2, 4, 6, 8]  # Wavelengths
betas = [1.5]  # Scaling factor for bandwidth
thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Orientations
psis = [0, np.pi]  # Phase offsets
gammas = [0.5]  # Aspect ratio
size = 128
gabor_filters = GaborFilters(size, lambdas,betas,thetas,psis,gammas)
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Extract the first 30 images
num_images = 2
dataset = train_images[:num_images]

# Normalize the dataset
dataset = dataset.astype(np.float32) / 255.0
neuron_inputs = generate_inputs_from_filters(dataset,
                                                gabor_filters,
                                                neuron_size = 14,
                                                image_size = 28,
                                                num_total_pixels=30,
                                                radius=1,
                                                shape="square")

Generating neuron mappings: 100%|██████████| 196/196 [00:04<00:00, 46.14it/s]


## Getting the Neural Network layer 1 sorted out
There are a few principle things to sort for layer 1:
Neurons 28x28
Interneurons (14x14)
poisson groups based on the input profile
tracking spikes & voltages.

I think I'll use clips rather than exponential curves.


In [17]:
from brian2 import *
from network.neurons import NeuronType
input_type = NeuronType("input")
inhibitory_type = NeuronType("inhibitory")
print("generating input layer")
first_layer = input_type.create_neuron_layer(size=14)
print("generating inhibitory layer")
first_inhibitory_layer = input_type.create_neuron_layer(size=7)

{'Cm': 0.5 * nfarad, 'g_leak': 25. * nsiemens, 'V_rest': -74. * mvolt, 'V_threshold': -53. * mvolt, 'V_reset': -57. * mvolt, 'V_reversal_e': 0. * volt, 'V_reversal_i': -70. * mvolt, 't_refract': 2. * msecond, 'sigma': 15. * uvolt, 'tau_m': 20. * msecond, 'tau_ee': 2. * msecond, 'tau_ie': 2. * msecond}
{'Cm': 214. * pfarad, 'g_leak': 28. * nsiemens, 'V_rest': -82. * mvolt, 'V_threshold': -53. * mvolt, 'V_reset': -58. * mvolt, 'V_reversal_e': 0. * volt, 'V_reversal_i': -70. * mvolt, 't_refract': 2. * msecond, 'sigma': 15. * uvolt, 'tau_m': 12. * msecond, 'tau_ei': 5. * msecond, 'tau_ii': 5. * msecond}
generating input layer
generating inhibitory layer


In [4]:
print(neuron_inputs.input_train[0])

[[1873.31121554  610.12062839 1906.65594093 5837.50297053 2000.69149329
  2556.64840686 2651.9995621  2017.29886095 2068.16301787 3380.94495084
  2724.6902103  1653.92892916 1685.53282681  910.9837632 ]
 [1713.59644786 4592.77447499 3226.84347595 1730.55867235 6236.36122085
  3395.19482243 3227.0497099  2201.44338951 4627.31274029 1938.56238256
  4348.67231533 5170.50934684 1321.35070817 3966.98148126]
 [2670.80807801 1957.64850443 3947.21034686 2317.79467382 3282.99855135
  4872.67169018 3802.77803973 5185.05426387 2329.82386049 3094.97351707
  1947.46065554 2762.78488494 2473.42716208 1916.51437364]
 [2338.31565959 4358.12511025 4218.53848416 4908.68327653 5979.15571548
  3988.72926143 5899.92616197 2130.56547895 3195.81410502 5566.82387577
  3350.29958292 3872.2536515  3085.2410895  4465.31154623]
 [4003.81419172 2032.13788264 7080.59908085 4154.16777847 5722.46422432
  1552.43657827 4262.76384473 5845.21979644 9811.2409803  3447.10795372
  3055.87078568 2478.15008315 5221.9972357  

In [18]:
from brian2 import *
import numpy as np

# Simulation parameters
grid_size = 14  # 14x14 grid
num_neurons = grid_size ** 2  # Total number of neurons in the input and first layer

# Generate a 14x14 input array (example: random values as input firing rates)
neuron_inputs_flat = neuron_inputs.input_train[0].flatten()

# Create a PoissonGroup to handle Poisson spike generation
poisson_group = PoissonGroup(num_neurons, rates=neuron_inputs_flat * Hz)
# Create the first layer of neurons, which will receive input from the Poisson neurons
# first_layer = NeuronGroup(num_neurons,
#                           '''
#                           ge : siemens
#                           x : integer (constant)
#                           y : integer (constant)
#                           ''')

# # Assign x and y coordinates to the first layer neurons
# first_layer.x = x_coords
# first_layer.y = y_coords
# Manually create x and y coordinates for the PoissonGroup
x_coords = np.array([i // grid_size for i in range(num_neurons)])
y_coords = np.array([i % grid_size for i in range(num_neurons)])


# Create synaptic connections where the x, y coordinates of both groups match
lambda_input = 0.4 * nsiemens
s_input = Synapses(poisson_group, first_layer, on_pre='ge_post += lambda_input')

# Now connect neurons where the x, y coordinates match
s_input.connect(condition='(i // grid_size == j // grid_size) and (i % grid_size == j % grid_size)')


WARNING    'i' is an internal variable of group 'synapses_1', but also exists in the run namespace with the value 6. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    'j' is an internal variable of group 'synapses_1', but also exists in the run namespace with the value 6. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    The object 'poissongroup' is getting deleted, but was never included in a network. This probably means that you did not store the object reference in a variable, or that the variable was not used to construct the network.
The object was created here (most recent call only):
  File '/var/folders/kt/dvpckm751r1656f50_b2ptl00000gn/T/ipykernel_55868/1481447788.py', line 12, in <module>
    poisson_group = PoissonGroup(num_neurons, rates=neuron_inputs_flat * Hz) [brian2.core.base.unused_brian_object]
WARNING    The object 'synapses' is getting deleted, but was never inc

Great, now let's connect the interneurons and then get the simulation up and running


In [19]:
# Simulation parameters
grid_size = 14  # 14x14 grid for the main layer
inhib_size = 7  # 7x7 grid for the inhibitory layer
num_neurons = grid_size ** 2  # Total number of neurons in the main layer
num_inhib_neurons = inhib_size ** 2  # Total number of neurons in the inhibitory layer
radius = 7  # Radius for fanning out connections

# Generate a 14x14 input array (example: random values as input firing rates)
np.random.seed(42)  # For reproducibility
neuron_inputs_flat = np.random.rand(num_neurons) * 20  # Random firing rates between 0 and 20 Hz

# Create the inhibitory neurons (7x7 grid)
inhib_layer = NeuronGroup(num_inhib_neurons,
                          '''
                          gi : siemens
                          x : integer (constant)
                          y : integer (constant)
                          ''',
                          threshold='gi > 1*nsiemens', reset='gi = 0*siemens')  # Define threshold for spiking

# Assign x and y coordinates to the inhibitory layer neurons
x_coords_inhib = np.array([i // inhib_size for i in range(num_inhib_neurons)])
y_coords_inhib = np.array([i % inhib_size for i in range(num_inhib_neurons)])
inhib_layer.x = x_coords_inhib
inhib_layer.y = y_coords_inhib

# Create synaptic connections where each neuron in the inhibitory layer
# connects to a fan-out set of neurons in the main layer based on distance
w_inhib = 0.4 * nsiemens  # Weight of the inhibitory synapse

# Synapse between the inhibitory layer and the main layer
syn_inhib = Synapses(inhib_layer, first_layer, on_pre='ge_post += w_inhib')

# Connect neurons based on topologically corresponding distances
for i in range(inhib_size):
    for j in range(inhib_size):
        # Center of the corresponding region in the main layer
        center_x = i * 2 + 1  # Middle of the 2x2 square in the main layer
        center_y = j * 2 + 1
        
        for m in range(grid_size):
            for n in range(grid_size):
                # Calculate the distance between the center and each neuron in the main layer
                dist = np.sqrt((center_x - m) ** 2 + (center_y - n) ** 2)
                
                # Connect if the neuron is within the specified radius
                if dist <= radius:
                    main_idx = m * grid_size + n  # Convert (m, n) to main layer neuron index
                    inhib_idx = i * inhib_size + j  # Convert (i, j) to inhibitory layer neuron index
                    
                    # Connect the inhibitory neuron to the corresponding main layer neurons
                    syn_inhib.connect(i=inhib_idx, j=main_idx)

# Define monitors for spikes
spike_mon_main = SpikeMonitor(first_layer)
spike_mon_inhib = SpikeMonitor(inhib_layer)

# Run the simulation
run(100 * ms)

# Print spike times to check the simulation
print("Main layer spikes:")
print(spike_mon_main.spike_trains())

print("\nInhibitory layer spikes:")
print(spike_mon_inhib.spike_trains())

WARNING    The object 'synapses_3' is getting deleted, but was never included in a network. This probably means that you did not store the object reference in a variable, or that the variable was not used to construct the network.
The object was created here (most recent call only):
  File '/var/folders/kt/dvpckm751r1656f50_b2ptl00000gn/T/ipykernel_55868/398250973.py', line 38, in <module>
    syn_inhib = Synapses(inhib_layer, first_layer, on_pre='ge_post += w_inhib') [brian2.core.base.unused_brian_object]
WARNING    The object 'neurongroup_2' is getting deleted, but was never included in a network. This probably means that you did not store the object reference in a variable, or that the variable was not used to construct the network.
The object was created here (most recent call only):
  File '/var/folders/kt/dvpckm751r1656f50_b2ptl00000gn/T/ipykernel_55868/398250973.py', line 19, in <module>
    inhib_layer = NeuronGroup(num_inhib_neurons, [brian2.core.base.unused_brian_object]


BrianObjectException: Error encountered with object named 'neurongroup_3_stateupdater'.
Object was created here (most recent call only, full details in debug log):
  File '/Users/jreid/Dropbox/dphil/programming/spikes/spikes/network/neurons.py', line 133, in create_neuron_layer
    neurons = NeuronGroup(

An error occurred when preparing an object. (See above for original error message and traceback.)

In [ ]:
S_input = Synapses(poisson_group,
                                G_e,
                                model='''
                    w : 1
                    dapre/dt = -apre/tau_pre : 1 (clock-driven)
                    dapost/dt = -apost/tau_post : 1 (clock-driven)
                    ''',
                    on_pre='''
                    ge_post += lambda_e * w
                    apre += alpha_C * (1-apre)
                    w += -w * apost * A_minus
                    ''',
                    on_post=f'''
                    apost += alpha_D * (1-apost)
                    w += (1-w) * apre * A_plus
                    '''
alpha_C = # synaptic_neurotransmitter_concentration

def create_input_neuron(neuron_parameters=None, model=None):
    if not model:
        model = """
            dv/dt = (V_rest-v)/tau_m + (ge*(V_reversal_e-v) + gi*(V_reversal_i-v))/(tau_m*g_leak) + sigma*xi*(tau_m)**-0.5 : volt
            dge/dt = -ge/tau_ee : siemens
            dgi/dt = -gi/tau_ie : siemens
            Cm : farad
            g_leak : siemens
            V_rest : volt
            V_reversal_e : volt
            V_reversal_i : volt
            sigma : volt
            tau_m : second
            tau_ee : second
            tau_ie : second
            x : integer (constant)
            y : integer (constant)
            """
    if not neuron_parameters:
        neuron_parameters = {
            "Cm": 214 * pF,  # Membrane capacitance
            "g_leak": 28 * nS,  # Leak conductance
            "V_rest": -82 * mV,  # Resting membrane potential
            "V_threshold": -53 * mV,  # Firing threshold
            "V_reset": -58 * mV,  # Reset potential after spike
            "V_reversal_e": 0 * mV,  # Excitatory reversal potential
            "V_reversal_i": -70 * mV,  # Inhibitory reversal potential
            "t_refract": 2 * ms,  # Refractory period
            "sigma": 0.015 * mV,  # Noise intensity
            "tau_m": 12 * ms,  # Membrane time constant
            "tau_ei": 5
            * ms,  # Synaptic time constant for excitatory synapses onto inhibitory neurons
            "tau_ii": 5
            * ms,  # Synaptic time constant for inhibitory synapses onto inhibitory neurons
        }


################# Defunct

In [2]:
from brian2 import *
from dataclasses import dataclass, field

# Define the data class with units, using default_factory for mutable types
@dataclass
class NeuronParameters:
    membrane_potential: Quantity = field(default_factory=lambda: 0 * mvolt)
    threshold: Quantity = field(default_factory=lambda: -50 * mvolt)
    resting_potential: Quantity = field(default_factory=lambda: -65 * mvolt)
    refractory_period: Quantity = field(default_factory=lambda: 5 * ms)
    time_constant: Quantity = field(default_factory=lambda: 10 * ms)

# Example usage
neuron_params = NeuronParameters(membrane_potential=-60*mvolt)
print(neuron_params)

NeuronParameters(membrane_potential=-60. * mvolt, threshold=-50. * mvolt, resting_potential=-65. * mvolt, refractory_period=5. * msecond, time_constant=10. * msecond)
